## 개념 짚고가기

- Seq2Seq(Encoder-Decoder LSTM)모델은 RNN을 이용해 input을 feature vector로 인코딩함
- 이렇게 인코딩된 vector를 여기선 'context vector'라 함
- 'context vector' : input문장의 추상적인/압축된 representation
- 이 context vector는 두번째 RNN을 통해 디코딩되어 번역된 문장을 생성함

In [15]:
# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd']
#       to   -> ['t', 'o', 'P', 'P']

input_batch, output_batch, target_batch = [], [], []
seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]
#print(len(seq_data))
n_step = 5
    
for seq in seq_data:
    # print(seq) # Output : one row
    for i in range(2):  # Check each one by one
       # print(seq)
       # print(n_step - len(seq[i]))
        seq[i] = seq[i] + 'P' * (n_step - len(seq[i])) # add 'P'
        print(seq[i])


manPP
women
black
white
kingP
queen
girlP
boyPP
upPPP
downP
highP
lowPP


In [16]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
num_dic = {n: i for i, n in enumerate(char_arr)}
num_dic

{'E': 1,
 'P': 2,
 'S': 0,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28}

In [17]:
input_batch, output_batch, target_batch = [], [], []
seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

n_step = 5
    
for seq in seq_data:
    # print(seq)  # Output : one row
    for i in range(2):  # Check each one by one
       # print(seq)
       # print(n_step - len(seq[i]))
        seq[i] = seq[i] + 'P' * (n_step - len(seq[i])) # add 'P'
        #print(seq[i])
    input = [num_dic[n] for n in seq[0]] # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 임베딩한다.
    print(input)

[15, 3, 16, 2, 2]
[4, 14, 3, 5, 13]
[13, 11, 16, 9, 2]
[9, 11, 20, 14, 2]
[23, 18, 2, 2, 2]
[10, 11, 9, 10, 2]


In [18]:
input_batch, output_batch, target_batch = [], [], []
seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]
num_dic = {n: i for i, n in enumerate(char_arr)}
n_step = 5
    
for seq in seq_data:
    # print(seq)  # Output : one row
    for i in range(2):  # Check each one by one
       # print(seq)
       # print(n_step - len(seq[i]))
        seq[i] = seq[i] + 'P' * (n_step - len(seq[i])) # add 'P'
        #print(seq[i])
    input = [num_dic[n] for n in seq[0]] # encoder
    output = [num_dic[n] for n in ('S' + seq[1])] # decoer
    target = [num_dic[n] for n in (seq[1] + 'E')] # training data
    print(target)

[25, 17, 15, 7, 16, 1]
[25, 10, 11, 22, 7, 1]
[19, 23, 7, 7, 16, 1]
[4, 17, 27, 2, 2, 1]
[6, 17, 25, 16, 2, 1]
[14, 17, 25, 2, 2, 1]


In [14]:
n_hidden = 128
batch_size = 6
hidden = torch.zeros(1, batch_size, n_hidden)
hidden

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0

---------------------------------------------------------------------------------------------

In [28]:
import torch
import torch.nn as nn

def make_batch(seq_data): 
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i])) # 학습데이터를 로드하여 char단위로 확인. n_step보다 짧을경우 P로 채움

        input = [num_dic[n] for n in seq[0]] # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        output = [num_dic[n] for n in ('S' + seq[1])] # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
        target = [num_dic[n] for n in (seq[1] + 'E')] # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E를 붙인다.

        input_batch.append(np.eye(n_class)[input]) # 단위행렬화
        output_batch.append(np.eye(n_class)[output])
        target_batch.append(target) # not one-hot

    # make tensor
    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)


In [3]:
# Model
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()

        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        enc_input = enc_input.transpose(0, 1) # enc_input: [max_len(=n_step, time step), batch_size, n_class]
        dec_input = dec_input.transpose(0, 1) # dec_input: [max_len(=n_step, time step), batch_size, n_class]

        # enc_states : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        _, enc_states = self.enc_cell(enc_input, enc_hidden)
        # outputs : [max_len+1(=6), batch_size, num_directions(=1) * n_hidden(=128)]
        outputs, _ = self.dec_cell(dec_input, enc_states)

        model = self.fc(outputs) # model : [max_len+1(=6), batch_size, n_class]
        return model

In [23]:
if __name__ == '__main__':
    n_step = 5
    n_hidden = 128

    char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
    num_dic = {n: i for i, n in enumerate(char_arr)}
    seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

    n_class = len(num_dic)
    batch_size = len(seq_data)
    model = Seq2Seq()
    criterion = nn.CrossEntropyLoss()  # loss function : CEE
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    input_batch, output_batch, target_batch = make_batch()

    for epoch in range(5000):
        # make hidden shape [num_layers * num_directions, batch_size, n_hidden]
        hidden = torch.zeros(1, batch_size, n_hidden)

        optimizer.zero_grad()
        # input_batch : [batch_size, max_len(=n_step, time step), n_class]
        # output_batch : [batch_size, max_len+1(=n_step, time step) (becase of 'S' or 'E'), n_class]
        # target_batch : [batch_size, max_len+1(=n_step, time step)], not one-hot
        output = model(input_batch, hidden, output_batch)
        # output : [max_len+1, batch_size, n_class]
        output = output.transpose(0, 1) # [batch_size, max_len+1(=6), n_class]
        loss = 0
        for i in range(0, len(target_batch)):
            # output[i] : [max_len+1, n_class, target_batch[i] : max_len+1]
            loss += criterion(output[i], target_batch[i])
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        loss.backward()
        optimizer.step()


C:\Users\ghdls\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 1000 cost = 0.003433
Epoch: 2000 cost = 0.000939
Epoch: 3000 cost = 0.000400
Epoch: 4000 cost = 0.000200
Epoch: 5000 cost = 0.000109


In [30]:
 # Test
def translate(word, args):
    input_batch, output_batch, _ = make_batch([[word, 'P' * len(word)]], args)

    # make hidden shape [num_layers * num_directions, batch_size, n_hidden]
    hidden = torch.zeros(1, 1, args.n_hidden)
    output = model(input_batch, hidden, output_batch)
    # output : [max_len+1(=6), batch_size(=1), n_class]

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    predict = output.data.max(2, keepdim=True)[1] # select n_class dimension
    decoded = [char_arr[i] for i in predict] # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    end = decoded.index('E') # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    translated = ''.join(decoded[:end])

    return translated.replace('P', '')

print('test')
print('man ->', translate('man'))
print('mans ->', translate('mans'))
print('king ->', translate('king'))
print('black ->', translate('black'))
print('upp ->', translate('upp'))


test


NameError: name 'args' is not defined